In [127]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta

In [128]:
def load_excel_to_dataframe(file_path, sheet_name=0):
    """
    Reads an Excel file and returns its contents as a pandas DataFrame.

    Parameters:
    - file_path (str): The path to the Excel file.
    - sheet_name (str or int, optional): The sheet name or index to read. Defaults to the first sheet (0).

    Returns:
    - DataFrame: The contents of the specified Excel sheet as a DataFrame.
    """
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name) # Reading excel sheet directly using pandas in to dataframe
        df = process_dataframe(df)
        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [129]:
def process_dataframe(df):
    df = df.rename(columns={'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'})
    return df

In [130]:
def aggregate_daily(df, agg_duration=15):
    '''
    Recieve dataframe df and aggregate it.

    Default aggregation duration (agg)duration is 15 minutes
    '''
    # Convert timestamp to date (removing time)
    df['date'] = df['timestamp'].dt.date

    # Group by date and aggregate every 15 mins
    df[f'{agg_duration}min'] = df['timestamp'].dt.floor(f'{agg_duration}min')
    grouped_data = df.groupby(['date', f'{agg_duration}min']).agg(
        open=('Open', 'first'),
        high=('High', 'max'),
        low=('Low', 'min'),
        close=('Close', 'last'),
        volume=('Volume', lambda x: x.fillna(0).sum())
    ).reset_index()

    return grouped_data

    # group by on a daily day to day basis
    # daily_data = df.groupby('date').agg(
    #     open=('o', 'first'),
    #     high=('h', 'max'),
    #     low=('l', 'min'),
    #     close=('c', 'last'),
    #     volume=('v', 'sum')
    # ).reset_index()
    #
    # return daily_data

In [131]:
def main():
    file_path = 'data/generated_data/SPY_Historical_Data_polygon_1minute_dur90d_2025-01-26_to_2025-02-02.xlsx'  # Replace with your actual file path
    file_path_reference = 'data/generated_data/SPY_Historical_Data_polygon_5minute_dur90d_2025-01-26_to_2025-02-02.xlsx'
    sheet_name = 'Sheet1'         # Replace with your actual sheet name or index
    agg_duration = 5 # set the aggregation duration
    df = load_excel_to_dataframe(file_path, sheet_name)
    df_reference = load_excel_to_dataframe(file_path_reference, sheet_name)

    # Read the first few rows off the dataframe
    if df is not None:
        print(df.head())
        print(df['Volume'].head().sum()) ## DEBUG: added for debugging volume sum in the aggregation process
    else:
        print('No data received')

    # Aggregate the minute data into 15min interval data
    df_agg_5min = aggregate_daily(df, agg_duration=5) # default of 15 mins
    print("\nFirst few rows of the aggregated data:\n", df_agg_5min.head(10)) # print the first few rows
    print(f"\nFirst few rows of the refernce {agg_duration}min data retrieved from Polygon.io:\n", df_reference.head(10)) # print the first few rows


In [132]:
if __name__ == '__main__':
    main()

            timestamp    Open    High     Low   Close  Volume
0 2025-01-27 09:00:00  598.78  598.83  595.90  596.00    8420
1 2025-01-27 09:01:00  596.05  596.22  595.84  596.14    3855
2 2025-01-27 09:02:00  596.18  596.24  596.02  596.02   14836
3 2025-01-27 09:03:00  596.05  596.05  595.68  595.85   13557
4 2025-01-27 09:04:00  595.76  595.90  595.76  595.90    1059
41727

First few rows of the aggregated data:
          date                5min    open    high     low   close  volume
0  2025-01-27 2025-01-27 09:00:00  598.78  598.83  595.68  595.90   83454
1  2025-01-27 2025-01-27 09:05:00  595.89  596.17  595.84  596.01   74762
2  2025-01-27 2025-01-27 09:10:00  596.16  596.53  595.97  596.00   18136
3  2025-01-27 2025-01-27 09:15:00  596.29  596.41  595.88  595.89   13068
4  2025-01-27 2025-01-27 09:20:00  595.98  596.49  595.98  596.40   16484
5  2025-01-27 2025-01-27 09:25:00  596.45  596.60  596.13  596.13   25112
6  2025-01-27 2025-01-27 09:30:00  596.05  596.54  595.89  596.